폴리백 수량계산
===

## 수량 계산과정

1. 오더번호에서 복종글자를 삭제한 항목을 만들고 리스트를 생성한다. (편의상 list_A 라고 호칭)
1. list_A 에서 항목을 하나씩 꺼낸다. (꺼낸 항목은 편의상 A라고 호칭)
    1. A는 3가지 경우로 나뉨 (오더번호가 3개, 2개 1개)
        1. 오더 3개의 경우
            1. H 복종의 지시량 * 1.02(LOSS)
            2. 남은 2복종의 1호 ~ 15호 세부수량을 비교하여 최대값을 누적
            3. 합계
        1. 오더 2개의 경우
            1. 두 오더의 1호 ~ 15호 세부수량을 비교하여 최대값을 누적
        1. 오더 1개의 경우
            1. 지시량 * 1.02(LOSS)

## ERP에서 자료 가져오기

In [1]:
from sqlalchemy import create_engine
import cx_Oracle
import pandas as pd
import binascii   # 한글 변환에 필요한 라이브러리
import sys
import math
import pyautogui
from datetime import datetime

In [2]:
sys.path.append('/settings')
import config

def connect_db(sid: str) -> 'sqlalchemy.engine.base.Engine':
    if sid != config.COMPANY_DB_CONFIG['sid']:
        raise ValueError("DB 를 찾을 수 없습니다.")
    
    conn = create_engine(
        "oracle+cx_oracle://{user}:{password}@{host}:{port}/{sid}?encoding=UTF-8&nencoding=UTF-8".format(
            user=config.COMPANY_DB_CONFIG['user'],
            password=config.COMPANY_DB_CONFIG['password'],
            host=config.COMPANY_DB_CONFIG['host'],
            port=config.COMPANY_DB_CONFIG['port'],
            sid=config.COMPANY_DB_CONFIG['sid']
        )
    )
    return conn

engine = connect_db('IVY')

## 사용자 입력값

In [3]:
confirm: str = pyautogui.confirm(title='알림', text='작업을 시작합니다.\
\n취소하면 작업을 중단합니다.\
\n\n< 기본적용 옵션 >\
\n1. 체육복 오더만 조회\
\n2. 대성섬유(VPF34), 동백섬유(VPF51) 2개 업체의 오더만 조회\
\n\n작업이 완료되면 엑셀파일을 확인하세요.')

if confirm == 'Cancel':
    sys.exit()

In [4]:
today: str = datetime.today().strftime("%Y%m%d")

In [5]:
start_date: str = pyautogui.prompt(title='조회시작일자', text=f'조회시작일자를 입력하세요. 예 : {today}', default=today)

In [6]:
# 오입력 검증
while True:
    i: int = 0
    if len(start_date) != 8:
        i += 1
        pyautogui.alert(title='입력오류', text='날짜는 8자리 입니다.')
        start_date = pyautogui.prompt(title='조회시작일자', text=f'조회시작일자를 입력하세요. 예 : {today}', default=today)

    if i == 0:
        break 

In [7]:
end_date: str = pyautogui.prompt(title='조회종료일자', text=f'조회종료일자를 입력하세요. 예 : {today}', default=today)

In [8]:
# 오입력 검증
while True:
    i: int = 0
    if len(start_date) != 8:
        i += 1
        pyautogui.alert(title='입력오류', text='날짜는 8자리 입니다.')
        end_date = pyautogui.prompt(title='조회종료일자', text=f'조회종료일자를 입력하세요. 예 : {today}', default=today)

    if i == 0:
        break 

In [9]:
start_date = start_date[:4] + '/' + start_date[4:6] + '/' + start_date[-2:]
end_date = end_date[:4] + '/' + end_date[4:6] + '/' + end_date[-2:]

start_date, end_date

('2022/07/01', '2022/07/25')

#### 한글변환 함수
    - SQL문의 한글 컬럼에는 다음과 같이 함수로 처리
    - rawtohex(utl_raw.cast_to_raw(변환할 컬럼))

In [10]:
# US7ASCII -> CP949(완성형한글) 로 변환
def us7ascii_to_cp949(df: pd.DataFrame) -> pd.DataFrame:
    for index, byte_data in enumerate(df):
        if byte_data == None: # null 값이면 패스. 안하면 변환 에러난다.
            continue
        byte_data = binascii.unhexlify(df[index])  # 16진수 문자열 hexstr로 표현된 바이너리 데이터를 반환. 역함수는 b2a_hex()
        df[index] = byte_data.decode("cp949")  # 바이트 변환값 -> cp949(완성형 한글) 로 변환
    return df

In [11]:
def count_polybag() -> pd.DataFrame:
    oracleSql: str = f'''
SELECT rawtohex(utl_raw.cast_to_raw(tkyk_name)) tkyk_name,
       rawtohex(utl_raw.cast_to_raw(agen_name)) agen_name,
       rawtohex(utl_raw.cast_to_raw(sch_name))sch_name,
       master_order,
       master_bokjong,
       Of_sojae_st_set(master_status,master_grade_cnt, b.bkjk_sojae_hgb, b.bkjk_sojae, b.bkjk_sojae_2h, b.bkjk_sojae_3h, s1.stand_sojae) sojae,
       To_char(Nvl(detail_qty1,0)) no_1,
       To_char(Nvl(detail_qty2,0)) no_2,
       To_char(Nvl(detail_qty3,0)) no_3,
       To_char(Nvl(detail_qty4,0)) no_4,
       To_char(Nvl(detail_qty5,0)) no_5,
       To_char(Nvl(detail_qty6,0)) no_6,
       To_char(Nvl(detail_qty7,0)) no_7,
       To_char(Nvl(detail_qty8,0)) no_8,
       To_char(Nvl(detail_qty9,0)) no_9,
       To_char(Nvl(detail_qty10,0)) no_10,
       To_char(Nvl(detail_qty11,0)) no_11,
       To_char(Nvl(detail_qty12,0)) no_12,
       To_char(Nvl(detail_qty13,0)) no_13,
       To_char(Nvl(detail_qty14,0)) no_14,
       To_char(Nvl(detail_qty15,0)) no_15,
       To_char(master_jisi_qty) jisi_qty,
       To_char(Nvl(jts_f_qty1,0) + Nvl(jts_f_qty2,0) + Nvl(jts_f_qty3,0) + Nvl(jts_f_qty4,0) + Nvl(jts_f_qty5,0) + Nvl(jts_t_qty1,0) + Nvl(jts_t_qty2,0) + Nvl(jts_t_qty3,0) + Nvl(jts_t_qty4,0) + Nvl(jts_t_qty5,0)) t_qty,
       To_char(fact_date,'yy/mm/dd') fact_date
FROM   i_suju_master_t,
       i_qty_detail_t,
       i_tkyk_t,
       i_agen_t,
       i_sch_t,
       i_suju_stand_t s1,
       i_stand_bkjk_t b,
       i_suju_fact_t
WHERE  master_status >= '50'
AND    master_tkyk = tkyk_code(+)
AND    master_agent = agen_code(+)
AND    master_school= sch_code(+)
AND    detail_gbn = '2'
AND    detail_order = master_order
AND    fact_order = master_order
AND    fact_code IN ('VPF34',
                     'VPF51')
AND    master_jaepum = 'F'
AND    b.bkjk_squota(+) = master_squota
AND    b.bkjk_school(+) = master_school
AND    b.bkjk_bokjong(+) = master_bokjong
AND    s1.stand_order(+) = master_order
AND    s1.stand_sojae_gbn(+)= '1'
AND    fact_date >= To_date('{start_date}','yyyy/mm/dd')
AND    fact_date <= To_date('{end_date}','yyyy/mm/dd')
ORDER BY master_order
'''
# AND    substr(fact_order,1,3) IN ('22F')
    df: pd.DataFrame = pd.read_sql_query(oracleSql, engine)
    df_temp: pd.DataFrame = df['tkyk_name'].copy()
    df['tkyk_name'] = us7ascii_to_cp949(df_temp)
    
    df_temp: pd.DataFrame = df['agen_name'].copy()
    df['agen_name'] = us7ascii_to_cp949(df_temp)
    
    df_temp: pd.DataFrame = df['sch_name'].copy()
    df['sch_name'] = us7ascii_to_cp949(df_temp)
    
    return df

In [12]:
df = count_polybag()

In [13]:
df.columns = ['특약', '대리점명', '학교명', '오더', '복종', '소재',
              '1호', '2호', '3호', '4호', '5호', '6호', '7호', '8호', '9호', '10호', '11호', '12호', '13호', '14호', '15호',
              '지시량', '특이수량', '타입일']
df['오더(비교용)'] = df['오더'].str[:8] + df['오더'].str[10:] # 비교용 컬럼

In [14]:
df.iloc[:, 6:23] = df.iloc[:, 6:23].astype(int) # 데이터 타입 일괄 변경
df.dtypes

특약         object
대리점명       object
학교명        object
오더         object
복종         object
소재         object
1호          int32
2호          int32
3호          int32
4호          int32
5호          int32
6호          int32
7호          int32
8호          int32
9호          int32
10호         int32
11호         int32
12호         int32
13호         int32
14호         int32
15호         int32
지시량         int32
특이수량        int32
타입일        object
오더(비교용)    object
dtype: object

In [15]:
poly_list = list(set(df['오더(비교용)'].tolist())) # 중복제거
poly_list

['23NAC1761',
 '22FAA6653',
 '22FTB0161',
 '23NHD0091',
 '22FJC0771',
 '22FAA3581',
 '22FBA0201',
 '22FNA0751',
 '22FUA1531',
 '22FTA0791',
 '22FUD0591',
 '22FAC2741',
 '22FAA9551',
 '22FAA6652',
 '22FAA9512',
 '22FBA1671',
 '22FTA1571',
 '22FAA5012',
 '22FAA5011',
 '22FAA7971',
 '22FSE0481',
 '22FTA1171',
 '22FNA1021',
 '22FPE0021',
 '22FLA0982',
 '22FNA0221',
 '22FBA1391',
 '22FAA7591',
 '22FRA0261',
 '22FAA7281',
 '22FSG0011',
 '22FBC0042',
 '22FAA0902',
 '22FKA1081',
 '22FPA1031',
 '22FSE0471',
 '22FAA0901',
 '22FHA0612',
 '22FNA0222',
 '22FTA1531',
 '22FAA6331',
 '22FTA0441',
 '22FSE0531',
 '22FBC0201',
 '22FPD0541',
 '22FSA2231',
 '22FTA0651',
 '22FPE0061',
 '22FTA1161',
 '22FAA6431',
 '22FAC0741',
 '22FAC8151',
 '22FAA0903',
 '22FNC1041',
 '23NAC4371',
 '22FAA6651',
 '22FHA1691',
 '22FPE0091',
 '22FAA1481',
 '22FUA0332',
 '22FTA1521']

In [16]:
poly_cnt: int = 0 # 반복문 돌며 각 세부수량 누적
poly_cnt_H: int = 0 # 3복종일때 H복종값 임시저장
df_total = pd.DataFrame()

for p_list in poly_list:
    df_temp = df[df['오더(비교용)'] == p_list].copy() # 그룹단위 반복문
    
    if len(df_temp) == 3:
        poly_cnt_H = math.ceil(df_temp[df_temp['복종'] == 'H']['지시량'] * 1.02) # H 복종 먼저 계산
        print('3복종 :', p_list, poly_cnt_H)
        for i in range(6, 21):
            poly_cnt = poly_cnt + df_temp[df_temp['복종'] != 'H'].iloc[:, i].max() # 2복종 세부수량에서 각 사이즈의 max 값을 누적합
        poly_cnt = math.ceil(poly_cnt * 1.02) # LOSS 적용값
        poly_cnt = poly_cnt + poly_cnt_H # 먼저 계산한 H 복종을 더해줌
        print('3복종 :', p_list, poly_cnt)
    elif len(df_temp) == 2:
        for i in range(6, 21):
            poly_cnt = poly_cnt + df_temp.iloc[:, i].max()
        poly_cnt = math.ceil(poly_cnt * 1.02)
        print('2복종 :', p_list, poly_cnt)
    elif len(df_temp) == 1:
        poly_cnt = math.ceil(df_temp['지시량'] * 1.02)
        print('1복종 :', p_list, poly_cnt)
    
    df_temp['복종수'] = len(df_temp)
    df_temp['폴리백'] = poly_cnt
    
    df_total = pd.concat([df_total, df_temp])
    
    poly_cnt = 0
    poly_cnt_H = 0

3복종 : 23NAC1761 266
3복종 : 23NAC1761 562
2복종 : 22FAA6653 94
3복종 : 22FTB0161 306
3복종 : 22FTB0161 847
2복종 : 23NHD0091 51
2복종 : 22FJC0771 21
3복종 : 22FAA3581 29
3복종 : 22FAA3581 215
2복종 : 22FBA0201 60
1복종 : 22FNA0751 49
1복종 : 22FUA1531 21
1복종 : 22FTA0791 175
2복종 : 22FUD0591 82
2복종 : 22FAC2741 27
2복종 : 22FAA9551 545
2복종 : 22FAA6652 68
1복종 : 22FAA9512 31
2복종 : 22FBA1671 158
1복종 : 22FTA1571 480
1복종 : 22FAA5012 56
1복종 : 22FAA5011 51
1복종 : 22FAA7971 11
1복종 : 22FSE0481 204
2복종 : 22FTA1171 498
1복종 : 22FNA1021 31
2복종 : 22FPE0021 451
1복종 : 22FLA0982 76
2복종 : 22FNA0221 51
2복종 : 22FBA1391 13
1복종 : 22FAA7591 31
1복종 : 22FRA0261 21
1복종 : 22FAA7281 31
2복종 : 22FSG0011 102
1복종 : 22FBC0042 11
2복종 : 22FAA0902 23
1복종 : 22FKA1081 59
1복종 : 22FPA1031 96
2복종 : 22FSE0471 388
3복종 : 22FAA0901 26
3복종 : 22FAA0901 49
2복종 : 22FHA0612 25
2복종 : 22FNA0222 159
1복종 : 22FTA1531 133
2복종 : 22FAA6331 45
1복종 : 22FTA0441 331
2복종 : 22FSE0531 368
2복종 : 22FBC0201 28
1복종 : 22FPD0541 21
1복종 : 22FSA2231 41
2복종 : 22FTA0651 148
1복종 : 22FPE0

In [17]:
df_total = df_total.sort_values(['오더(비교용)', '복종'])
df_total

,특약,대리점명,학교명,오더,복종,소재,1호,2호,3호,4호,...,12호,13호,14호,15호,지시량,특이수량,타입일,오더(비교용),복종수,폴리백
0,중부상권,일산점,백석중(일산),22FAA090F 1,F,HCR0058,0,0,0,2,...,0,0,0,0,22,0,22/07/06,22FAA0901,3,49
3,중부상권,일산점,백석중(일산),22FAA090H 1,H,HCM0991,0,0,2,6,...,0,0,0,0,25,0,22/07/06,22FAA0901,3,49
4,중부상권,일산점,백석중(일산),22FAA090W 1,W,HCR0058,0,0,0,2,...,0,0,0,0,22,0,22/07/06,22FAA0901,3,49
1,중부상권,일산점,백석중(일산),22FAA090F 2,F,HCR0058,0,0,0,0,...,0,0,0,0,22,0,22/07/13,22FAA0902,2,23
5,중부상권,일산점,백석중(일산),22FAA090W 2,W,HCR0058,0,0,0,0,...,0,0,0,0,22,0,22/07/13,22FAA0902,2,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,중부상권,분당점,계원예고,23NAC176W 1,W,HCM0580,0,2,40,84,...,0,0,0,0,260,0,22/07/16,23NAC1761,3,562
96,중부상권,만안점,양명고(학년별-2줄),23NAC437F 1,F,HCR0919,0,0,0,0,...,0,0,0,0,314,0,22/07/13,23NAC4371,2,321
97,중부상권,만안점,양명고(학년별-2줄),23NAC437W 1,W,HCR0919,0,0,0,0,...,0,0,0,0,298,0,22/07/13,23NAC4371,2,321
98,대전상권,논산점,논산여고,23NHD009F 1,F,HCR0580,0,0,2,6,...,2,0,0,0,50,0,22/07/06,23NHD0091,2,51


In [18]:
df_total = df_total.set_index(['오더(비교용)', '폴리백', '복종수', '특약', '대리점명', '학교명', '소재'])

In [19]:
df_total = df_total.reset_index('오더(비교용)', drop=True)

In [20]:
df_total.to_excel(f'{start_date.replace("/", "")}_{end_date.replace("/", "")}_폴리백수량.xlsx')

In [21]:
pyautogui.alert(title='알림', text='작업이 종료되었습니다.\n엑셀파일을 확인하세요.')

'OK'